In [1]:
import numpy as np
from numpy import nan

from tabulate import tabulate
import random
from itertools import zip_longest
from collections import namedtuple
from stack import Stack
from actions import Moove, Action, List_actions
from utils_class import EpsilonGreedy, ReplayMemory, Agent, Experience
from DQN import DQNAgent    

%load_ext autoreload
%autoreload 2


2022-05-25 08:44:53.710582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 08:44:53.710608: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
STACK_SIZE = 10

EPISODE = 2500
CAPACITY = 100
BATCH_SIZE = 64
MIN_REPLAY_MEMORY_SIZE = 100

DECAY = 0.998
MAX_EPSILON = 0.99
MIN_EPSILON = 0.15

ACTION_REWARD = -1
INVERSE_ACTION_REWARD = -5
FINISH_REWARD = 100

LEARNING_RATE = 0.1
DISCOUNT = 0.95
GAMMA = 0.99


In [9]:
# Experience = namedtuple(
#                 'Experience', ('state', 'action', 'next_state', 'reward'))
class Env:
    
    
    def __init__(self, size):
        self.A = Stack(size, 0)
        self.B = Stack(size, 1)
        self.size = size
        self.state_t = np.vstack((self.A.stack, self.B.stack))
        self.state_t1 = np.vstack((self.A.stack, self.B.stack))
        
        self.moover = Moove(self.A, self.B, size)
        self.actions = Action(self.A, self.B)
        self.liste = List_actions()
        self.current_action = 1;
        # self.agent = Agent(EpsilonGreedy(MIN_EPSILON, MAX_EPSILON, DECAY), 11)
        self.strategy = EpsilonGreedy(MIN_EPSILON, MAX_EPSILON, DECAY)
        self.agent = DQNAgent(size)
        
        self.cum_reward = 0
        self.current_reward = 0
        self.step = 0
        
        self.exp = Experience()
        self.experience = None
        self.replaymemory = ReplayMemory(CAPACITY)
    
    def reset(self, size):
        self.A = Stack(size)
        self.B = Stack(0)
        
    def state(self):
        return np.vstack((self.A.stack, self.B.stack))
        
    def actions_available(self):
        return self.actions.possible_actions()
    
    def reward(self):
        if self.B.stack.size == 0 and np.all(self.A.stack[:-1] <= self.A.stack[1:]):
            self.cum_reward += 100
        
        return (self.cum_reward)
        
    def choose_action(self):
        exploration_rate = self.strategy.get_exploration_rate(self.step)
        
        self.step += 1
        
        if (exploration_rate > random.random()):
            #exploration
            self.current_action = np.random.choice(self.actions.possible_actions())
            return self.current_action
        else :
            self.current_action = np.argmax(self.agent.policy_model.predict(self.state))
            return self.current_action
    
    def take_actions(self):
        
        self.state_t = self.state()
        
        self.liste.add(self.current_action)
        self.current_reward += self.liste.compare()
        
        if self.current_action == 0:
            self.current_reward += self.moover.push_b(self.A, self.B)
            
        elif self.current_action == 1:
            self.current_reward += self.moover.push_a(self.A, self.B)
            
        elif self.current_action == 2:
            self.current_reward += self.moover.rotate_b(self.B)
            
        elif self.current_action == 3:
            self.current_reward += self.moover.rotate_a(self.A)
            
        elif self.current_action == 4:
            self.current_reward += self.moover.inverse_rotate_b(self.B)
            
        elif self.current_action == 5:
            self.current_reward += self.moover.inverse_rotate_a(self.A)
            
        elif self.current_action == 6:
           self.current_reward += self.moover.swap_b(self.B)
            
        elif self.current_action == 7:
            self.current_reward += self.moover.swap_a(self.A)
            
        elif self.current_action == 8:
           self.current_reward += self.moover.rotate(self.A, self.B)
            
        elif self.current_action == 9:
            self.current_reward += self.moover.reverse(self.A, self.B)
            
        elif self.current_action == 10:
            self.current_reward += self.moover.swap(self.A, self.B)
        
        self.state_t1 = self.state()

    def action_space(self):
      return np.array([0,
                       1,
                       2,
                       3,
                       4,
                       5,
                       6,
                       7,
                       8,
                       9,
                       10])
      
    def print_state(self):
        print("---State t---")
        print("-A-\t-B-")
        print("___________\t")
        for a, b in zip_longest(reversed(self.A.stack), reversed(self.B.stack), fillvalue='xxx'):
          
          print(f"|{a}\t{b}|")
        print("___________\t")
    
    def create_experience(self):
        self.experience = self.exp.create_experience(self.state_t, self.current_action, self.state_t1, self.current_reward)
        self.cum_reward += self.current_reward
        self.current_reward = 0
        
    def print_experience(self):
        A_t, B_t = np.split(self.experience.state, 2, axis=0)
        A_t1, B_t1 = np.split(self.experience.next_state, 2, axis=0)
        
        A_t = A_t.reshape((A_t.shape[1]))
        B_t = B_t.reshape((B_t.shape[1]))
        
        A_t1 = A_t1.reshape((A_t1.shape[1]))
        B_t1 = B_t1.reshape((B_t1.shape[1]))
        
        print("---State t---\t\t---State t+1---")
        print("-A-\t-B-\t\t-A-\t-B-")
        print("___________\t\t___________")
        for a_t, b_t, a_t1, b_t1 in zip_longest(reversed(A_t), reversed(B_t), reversed(A_t1), reversed(B_t1)):
          
          print(f"|{a_t}\t{b_t}|\t\t|{a_t1}\t{b_t1}|")
        print("___________\t\t___________")
        print("----------------------------------")
        print(f"\tAction [{self.experience.action}]")
        print(f"\tReward [{self.experience.reward}]")
        print("----------------------------------")
    


In [10]:
size = 10
env = Env(size)

In [11]:
# env.print_stack()
# env.moover.push_b()
# env.choose_action()
env.current_action = 0
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.current_action = 1
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.current_action = 0
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
env.take_actions()
env.create_experience()
env.replaymemory.push(env.experience)
# print(env.experience)
env.print_state()


---State t---
-A-	-B-
___________	
|52.0	4.0|
|15.0	92.0|
|48.0	nan|
|68.0	nan|
|8.0	nan|
|89.0	nan|
|78.0	nan|
|60.0	nan|
|nan	nan|
|nan	nan|
___________	


In [6]:
env.replaymemory.get_sample(4)

[Experience(state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), action=1, next_state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), reward=-6),
 Experience(state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), action=0, next_state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), reward=-1),
 Experience(state=array([[13., 27., 84., 58., 93., 67., 35., 11., 71., 26.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32), action=0, next_state=array([[nan, 13., 27., 84., 58., 93., 67., 35., 11., 71.],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.]], dtype=float32), reward=-6),
 Experience(state=array([[nan, 

In [7]:
env.replaymemory.print_history(BATCH_SIZE)

Actions : 0 	 Reward : -1
Actions : 1 	 Reward : -6
Actions : 0 	 Reward : -6
Actions : 0 	 Reward : -1
